In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer

# plotting convenience
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style('whitegrid')

# load data
df = pd.read_csv('../data/sierraleone-bumbuna.csv', parse_dates=['Timestamp'])
print(f'Data shape: {df.shape}')
df.head()

# Sierraleone - Exploratory Data Analysis
## 1. Summary Statistics & Missing Value Report

In [ ]:
# Summary statistics
display(df.describe(include='all').T)

# Missing values report
missing = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)
missing_df = pd.concat([missing, missing_pct], axis=1, keys=['missing_count', 'missing_pct'])
print('\nMissing Values Report:')
display(missing_df[missing_df['missing_count'] > 0])

# Columns with >5% missing
high_missing = missing_df[missing_df['missing_pct'] > 5]
if len(high_missing) > 0:
    print('\nColumns with >5% missing values:')
    display(high_missing)
else:
    print('\nNo columns with >5% missing values.')

## 2. Outlier Detection & Basic Cleaning

In [ ]:
# Compute Z-scores for key columns
num_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
sub = df[num_cols].copy()

# Compute z-scores
z = np.abs(stats.zscore(sub.dropna()))
z = pd.DataFrame(z, columns=sub.dropna().columns, index=sub.dropna().index)

# Flag rows with |Z| > 3
outlier_mask = (z > 3).any(axis=1)
outliers = df.loc[outlier_mask, ['Timestamp'] + num_cols]
print(f'Number of outlier rows (|Z| > 3): {len(outliers)}')
if len(outliers) > 0:
    print('\nSample outliers:')
    display(outliers.head(10))

In [ ]:
# Create a copy for cleaning
df_clean = df.copy()

# Replace impossible values (negative irradiance values are not physically possible)
df_clean.loc[df_clean['GHI'] < 0, 'GHI'] = np.nan
df_clean.loc[df_clean['DNI'] < 0, 'DNI'] = np.nan
df_clean.loc[df_clean['DHI'] < 0, 'DHI'] = np.nan

# Median imputer for chosen columns
imputer = SimpleImputer(strategy='median')
df_clean[num_cols] = imputer.fit_transform(df_clean[num_cols])

# Set Timestamp as index for time-based interpolation
df_clean_indexed = df_clean.set_index('Timestamp')

# Linear interpolation for TModA and TModB
df_clean_indexed['TModA'] = df_clean_indexed['TModA'].interpolate(method='linear', limit=6)
df_clean_indexed['TModB'] = df_clean_indexed['TModB'].interpolate(method='linear', limit=6)

# Reset index
df_clean = df_clean_indexed.reset_index()

# Drop Comments column if it's all NaN
if 'Comments' in df_clean.columns and df_clean['Comments'].isna().all():
    df_clean = df_clean.drop(columns=['Comments'])

print(f'Cleaned data shape: {df_clean.shape}')
print(f'Remaining missing values: {df_clean.isna().sum().sum()}')

## 3. Time Series Analysis

In [ ]:
# Line charts of GHI, DNI, DHI, Tamb vs. Timestamp
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Sample data for visualization (every 100th point for performance)
df_sample = df_clean.set_index('Timestamp').resample('1H').mean()

axes[0, 0].plot(df_sample.index, df_sample['GHI'], linewidth=0.5, alpha=0.7)
axes[0, 0].set_title('GHI Over Time')
axes[0, 0].set_ylabel('GHI (W/m²)')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(df_sample.index, df_sample['DNI'], linewidth=0.5, alpha=0.7, color='orange')
axes[0, 1].set_title('DNI Over Time')
axes[0, 1].set_ylabel('DNI (W/m²)')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(df_sample.index, df_sample['DHI'], linewidth=0.5, alpha=0.7, color='green')
axes[1, 0].set_title('DHI Over Time')
axes[1, 0].set_ylabel('DHI (W/m²)')
axes[1, 0].set_xlabel('Timestamp')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(df_sample.index, df_sample['Tamb'], linewidth=0.5, alpha=0.7, color='red')
axes[1, 1].set_title('Ambient Temperature Over Time')
axes[1, 1].set_ylabel('Tamb (°C)')
axes[1, 1].set_xlabel('Timestamp')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Monthly patterns
df_clean['Month'] = pd.to_datetime(df_clean['Timestamp']).dt.month
df_clean['Hour'] = pd.to_datetime(df_clean['Timestamp']).dt.hour

# Monthly averages
monthly_avg = df_clean.groupby('Month')[['GHI', 'DNI', 'DHI', 'Tamb']].mean()

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
monthly_avg['GHI'].plot(kind='bar', ax=axes[0, 0], color='skyblue')
axes[0, 0].set_title('Average GHI by Month')
axes[0, 0].set_ylabel('GHI (W/m²)')
axes[0, 0].set_xlabel('Month')

monthly_avg['DNI'].plot(kind='bar', ax=axes[0, 1], color='orange')
axes[0, 1].set_title('Average DNI by Month')
axes[0, 1].set_ylabel('DNI (W/m²)')
axes[0, 1].set_xlabel('Month')

monthly_avg['DHI'].plot(kind='bar', ax=axes[1, 0], color='green')
axes[1, 0].set_title('Average DHI by Month')
axes[1, 0].set_ylabel('DHI (W/m²)')
axes[1, 0].set_xlabel('Month')

monthly_avg['Tamb'].plot(kind='bar', ax=axes[1, 1], color='red')
axes[1, 1].set_title('Average Temperature by Month')
axes[1, 1].set_ylabel('Tamb (°C)')
axes[1, 1].set_xlabel('Month')

plt.tight_layout()
plt.show()

In [ ]:
# Hourly patterns
hourly_avg = df_clean.groupby('Hour')[['GHI', 'Tamb']].mean()

fig, axes = plt.subplots(1, 2, figsize=(16, 5))
hourly_avg['GHI'].plot(kind='line', ax=axes[0], marker='o', color='skyblue')
axes[0].set_title('Average GHI by Hour of Day')
axes[0].set_ylabel('GHI (W/m²)')
axes[0].set_xlabel('Hour')
axes[0].grid(True, alpha=0.3)

hourly_avg['Tamb'].plot(kind='line', ax=axes[1], marker='o', color='red')
axes[1].set_title('Average Temperature by Hour of Day')
axes[1].set_ylabel('Tamb (°C)')
axes[1].set_xlabel('Hour')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Cleaning Impact Analysis

In [ ]:
# Group by Cleaning flag and plot average ModA & ModB
cleaning_impact = df_clean.groupby('Cleaning')[['ModA', 'ModB']].mean()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
cleaning_impact.plot(kind='bar', ax=axes[0])
axes[0].set_title('Average ModA & ModB by Cleaning Status')
axes[0].set_ylabel('Irradiance (W/m²)')
axes[0].set_xlabel('Cleaning (0=No, 1=Yes)')
axes[0].legend(['ModA', 'ModB'])
axes[0].grid(True, alpha=0.3, axis='y')

# Before/After comparison
before_clean = df_clean[df_clean['Cleaning'] == 0][['ModA', 'ModB']].mean()
after_clean = df_clean[df_clean['Cleaning'] == 1][['ModA', 'ModB']].mean()

comparison = pd.DataFrame({'Before Cleaning': before_clean, 'After Cleaning': after_clean})
comparison.plot(kind='bar', ax=axes[1])
axes[1].set_title('ModA & ModB: Before vs After Cleaning')
axes[1].set_ylabel('Irradiance (W/m²)')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('Cleaning Impact Summary:')
display(cleaning_impact)

## 5. Correlation & Relationship Analysis

In [ ]:
# Heatmap of correlations
corr_cols = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB', 'Tamb', 'RH', 'WS', 'BP']
corr_matrix = df_clean[corr_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={'shrink': 0.8})
plt.title('Correlation Heatmap - Key Variables')
plt.tight_layout()
plt.show()

In [ ]:
# Scatter plots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Sample for performance
df_scatter = df_clean.sample(min(10000, len(df_clean)))

axes[0, 0].scatter(df_scatter['WS'], df_scatter['GHI'], alpha=0.3, s=10)
axes[0, 0].set_xlabel('Wind Speed (m/s)')
axes[0, 0].set_ylabel('GHI (W/m²)')
axes[0, 0].set_title('WS vs GHI')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].scatter(df_scatter['WSgust'], df_scatter['GHI'], alpha=0.3, s=10, color='orange')
axes[0, 1].set_xlabel('Wind Gust Speed (m/s)')
axes[0, 1].set_ylabel('GHI (W/m²)')
axes[0, 1].set_title('WSgust vs GHI')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].scatter(df_scatter['WD'], df_scatter['GHI'], alpha=0.3, s=10, color='green')
axes[1, 0].set_xlabel('Wind Direction (°)')
axes[1, 0].set_ylabel('GHI (W/m²)')
axes[1, 0].set_title('WD vs GHI')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].scatter(df_scatter['RH'], df_scatter['Tamb'], alpha=0.3, s=10, color='red')
axes[1, 1].set_xlabel('Relative Humidity (%)')
axes[1, 1].set_ylabel('Ambient Temperature (°C)')
axes[1, 1].set_title('RH vs Tamb')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# RH vs GHI
plt.figure(figsize=(10, 6))
df_scatter = df_clean.sample(min(10000, len(df_clean)))
plt.scatter(df_scatter['RH'], df_scatter['GHI'], alpha=0.3, s=10)
plt.xlabel('Relative Humidity (%)')
plt.ylabel('GHI (W/m²)')
plt.title('RH vs GHI')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Wind & Distribution Analysis

In [ ]:
# Wind direction distribution (simplified wind rose)
wind_bins = np.arange(0, 370, 30)
wind_labels = [f'{i}-{i+30}°' for i in range(0, 360, 30)]
df_clean['WD_bin'] = pd.cut(df_clean['WD'], bins=wind_bins, labels=wind_labels, include_lowest=True)

wind_dir_counts = df_clean['WD_bin'].value_counts().sort_index()

plt.figure(figsize=(10, 8))
angles = np.linspace(0, 2 * np.pi, len(wind_dir_counts), endpoint=False).tolist()
angles += angles[:1]  # Complete the circle

values = wind_dir_counts.values.tolist()
values += values[:1]

ax = plt.subplot(111, projection='polar')
ax.plot(angles, values, 'o-', linewidth=2)
ax.fill(angles, values, alpha=0.25)
ax.set_xticks(angles[:-1])
ax.set_xticklabels(wind_dir_counts.index)
ax.set_title('Wind Direction Distribution (Radial Plot)', pad=20)
plt.show()

In [ ]:
# Histograms
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

axes[0].hist(df_clean['GHI'], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].set_xlabel('GHI (W/m²)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('GHI Distribution')
axes[0].grid(True, alpha=0.3, axis='y')

axes[1].hist(df_clean['WS'], bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1].set_xlabel('Wind Speed (m/s)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Wind Speed Distribution')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 7. Temperature Analysis

In [ ]:
# Examine how RH influences temperature and solar radiation
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

df_scatter = df_clean.sample(min(10000, len(df_clean)))

scatter1 = axes[0].scatter(df_scatter['RH'], df_scatter['Tamb'], 
                         c=df_scatter['GHI'], cmap='viridis', 
                         alpha=0.5, s=20)
axes[0].set_xlabel('Relative Humidity (%)')
axes[0].set_ylabel('Ambient Temperature (°C)')
axes[0].set_title('RH vs Tamb (colored by GHI)')
plt.colorbar(scatter1, ax=axes[0], label='GHI (W/m²)')
axes[0].grid(True, alpha=0.3)

scatter2 = axes[1].scatter(df_scatter['RH'], df_scatter['GHI'], 
                         c=df_scatter['Tamb'], cmap='coolwarm', 
                         alpha=0.5, s=20)
axes[1].set_xlabel('Relative Humidity (%)')
axes[1].set_ylabel('GHI (W/m²)')
axes[1].set_title('RH vs GHI (colored by Tamb)')
plt.colorbar(scatter2, ax=axes[1], label='Tamb (°C)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Bubble Chart: GHI vs Tamb

In [ ]:
# Bubble chart: GHI vs Tamb with bubble size = RH
df_bubble = df_clean.sample(min(5000, len(df_clean)))

plt.figure(figsize=(12, 8))
scatter = plt.scatter(df_bubble['Tamb'], df_bubble['GHI'], 
                     s=df_bubble['RH']*2, alpha=0.5, 
                     c=df_bubble['BP'], cmap='plasma')
plt.xlabel('Ambient Temperature (°C)')
plt.ylabel('GHI (W/m²)')
plt.title('GHI vs Tamb (Bubble size = RH, Color = BP)')
plt.colorbar(scatter, label='Barometric Pressure (hPa)')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Export Cleaned Data

In [ ]:
# Export cleaned DataFrame
df_clean.drop(columns=['Month', 'Hour', 'WD_bin'], errors='ignore').to_csv('../data/sierraleone_clean.csv', index=False)
print('Cleaned data exported to ../data/sierraleone_clean.csv')
print('Final shape:', df_clean.shape)